In [ ]:
# Import necessary libraries
import os
import cv2
import numpy as np
from google.colab import files
from zipfile import ZipFile
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import StandardScaler

# uploaded = files.upload()
# for zip_file in uploaded.keys():
#     with ZipFile(zip_file, 'r') as zip_ref:
#         folder_name = zip_file.split('.')[0]
#         zip_ref.extractall(folder_name)







In [ ]:
normal_mri_path = "/content/Normal"
parkinson_mri_path = "/content/Parkinson"

img_rows, img_cols = 128, 128

In [ ]:
def load_images(folder, label):
    images = []
    labels = []
    for file in os.listdir(folder):
        if file.endswith(".jpg") or file.endswith(".png"):
            img_path = os.path.join(folder, file)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, (img_rows, img_cols))
            images.append(img.flatten())
            labels.append(label)
    return np.array(images), np.array(labels)

In [ ]:
normal_images, normal_labels = load_images(normal_mri_path, 0)
parkinson_images, parkinson_labels = load_images(parkinson_mri_path, 1)

In [ ]:

X = np.vstack((normal_images, parkinson_images))
y = np.hstack((normal_labels, parkinson_labels))

scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

svm = SVC(kernel='linear', C=1.0, random_state=42)
svm.fit(X_train, y_train)

SVC(kernel='linear', random_state=42)

In [ ]:
y_pred = svm.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.9820359281437125
Classification Report:
               precision    recall  f1-score   support

           0       0.97      1.00      0.99       112
           1       1.00      0.95      0.97        55

    accuracy                           0.98       167
   macro avg       0.99      0.97      0.98       167
weighted avg       0.98      0.98      0.98       167



In [ ]:
def preprocess_image(image_path):
    """
    Preprocess a single image for prediction.
    :param image_path: Path to the MRI scan image.
    :return: Preprocessed and flattened image array.
    """
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        raise FileNotFoundError(f"Image at {image_path} not found.")
    IMAGE_SIZE = (img_rows, img_cols)
    img = cv2.resize(img, IMAGE_SIZE)
    flattened_img = img.flatten().reshape(1, -1)
    return flattened_img

def predict_image(image_path, model):
    """
    Predict the class of a single image using the trained SVM model.
    :param image_path: Path to the MRI scan image.
    :param model: Trained SVM model.
    :return: Predicted class label (0 for normal, 1 for Parkinson's).
    """
    processed_img = preprocess_image(image_path)
    prediction = model.predict(processed_img)
    return prediction[0]

test_image_path = "/content/t1_fl3d_sag_p4_iso_1.0_045.png"
try:
    prediction = predict_image(test_image_path, svm)
    if prediction == 0:
        print(f"The MRI scan is predicted as: Normal")
    else:
        print(f"The MRI scan is predicted as: Parkinson's")
except FileNotFoundError as e:
    print(e)


The MRI scan is predicted as: Normal


In [ ]:
import joblib

joblib.dump(svm, 'svm_model.pkl')
joblib.dump(scaler, 'scaler.pkl')
print("Model and scaler saved successfully!")

Model and scaler saved successfully!
